In [ ]:
using Pkg
Pkg.add("Plots")
Pkg.add("Random")
Pkg.add("GLM")
Pkg.add("DataFrames")
Pkg.add("LaTeXStrings")
Pkg.add("Statistics") 
Pkg.add("StatsBase")
Pkg.add("CSV")
Pkg.add("CategoricalArrays")

In [ ]:
#3 Real Data
using DataFrames
using CategoricalArrays
using Plots
using Random
using GLM
using DataFrames
using LaTeXStrings
using Statistics
using StatsBase
using CSV

#3a Cleaning
df=CSV.read("apartments.csv",DataFrame)
df[!,"area2"]=df[:,"area"].^2
df.hasparkingspace = ifelse.(df.hasparkingspace .== "yes", 1, 0)
df.hasbalcony = ifelse.(df.hasbalcony .== "yes", 1, 0)
df.hassecurity = ifelse.(df.hassecurity .== "yes", 1, 0)
df.hasstorageroom = ifelse.(df.hasstorageroom .== "yes", 1, 0)
df.haselevator = passmissing(x -> x == "yes" ? 1 : 0).(df.haselevator)
for i in 0:9
    df[!, Symbol("end_$i")]=ifelse.(df.area.% 10 .==i,1,0)
end 
CSV.write(" Clean dataset.csv", df)

# 3b Linear model estimation 

df.month = categorical(df.month)
df.type = categorical(df.type)
df.rooms = categorical(df.rooms)
df.ownership = categorical(df.ownership)
df.buildingmaterial = categorical(df.buildingmaterial)

regresores=names(df)[3:31]
results=lm(term(:price)~sum(term.(regresores)), df)
CSV.write(" Linear model estimation.csv", coeftable(results))
#Parece que el hecho de que el área de un departamento termine en 0  aumenta el precio de este en aproximadamente 26904.7 a comparacion de si terminara en 9 (la categoria base). Este  coeficiente estimado parece significativo por el valor tan bajo del p-value

# partialling-out
Z=names(df)[3:31]
Z=setdiff(Z, ["end_0"])
price_Z_model=lm(term(:price)~sum(term.(Z)), df)
price_Z_residual=residuals(price_Z_model)
end_0_Z_model=lm(term(:end_0)~sum(term.(Z)), df)
end_0_Z_residual=residuals(end_0_Z_model)
results_partialling_out=lm([end_0_Z_residual;;], price_Z_residual)
CSV.write(" Partiall-out estimation.csv", coeftable(results_partialling_out))

#3c Price premium for area that ends in 0-digit 
#1 Train the model
train_df=df[df[:,"end_0"].!=1,:]
regresores=names(df)[3:31]
train_model=lm(term(:price)~sum(term.(regresores)), train_df)
println(coeftable(train_model))

#2 Predict prices
price_prediction=predict(train_model,select(df,regresores))

#3 Compare averages
df_predicted_price=deepcopy(df)
df_predicted_price.predicted_price=price_prediction
df_end_0=df_predicted_price[df_predicted_price[:,"end_0"].==1,:]
average_actual_price=mean(df_end_0[:,"price"])
average_predicted_price=mean(skipmissing(df_end_0[:,"predicted_price"]))
df_average_prices=DataFrame(average_actual_price=[average_actual_price],average_predicted_price=[average_predicted_price])
CSV.write("Average prices for apartments with areas ending in 0.csv",df_average_prices)
average_actual_price-average_predicted_price
#El precio promedio real para los departamentos con área que termina en 0 es de 873616 aproximadamente, mientras que el modelo predice que el área deberia ser 877539. Es decir, los departamentos se venden a un menor precio a comparación de lo que predice el modelo; sin embargo, la diferencia es de solo 3923.
#Esta diferencia en precios no parece importante tomando en cuenta las cifras de los precios promedio.